In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
(mnist_train, mnist_test), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [5]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

def transform_labels(image, label):
  return image, tf.math.floor(label / 2)

def prepare(ds, shuffle=True, batch_size=32, prefetch=True):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.map(transform_labels, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.shuffle(ds_info.splits['train'].num_examples) if shuffle else ds
  ds = ds.cache()
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE) if prefetch else ds
  return ds

def split_tasks(ds, predicate):
  return ds.filter(predicate), ds.filter(lambda img, label: not predicate(img, label))

multi_task_train, multi_task_test = prepare(mnist_train), prepare(mnist_test)
task_A_train, task_B_train = split_tasks(mnist_train, lambda img, label: label % 2 == 0)
task_A_train, task_B_train = prepare(task_A_train), prepare(task_B_train)
task_A_test, task_B_test = split_tasks(mnist_test, lambda img, label: label % 2 == 0)
task_A_test, task_B_test = prepare(task_A_test), prepare(task_B_test)

In [6]:
def evaluate(model, test_set):
  acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
  for i, (imgs, labels) in enumerate(test_set):
    preds = model.predict_on_batch(imgs)
    acc.update_state(labels, preds)
  return acc.result().numpy()

In [33]:
multi_task_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

multi_task_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

multi_task_model.fit(multi_task_train, epochs=6)

Epoch 1/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2241 - accuracy: 0.9306
Epoch 2/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1013 - accuracy: 0.9690
Epoch 3/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0723 - accuracy: 0.9779
Epoch 4/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0550 - accuracy: 0.9835
Epoch 5/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0419 - accuracy: 0.9878
Epoch 6/6
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0319 - accuracy: 0.9910


In [34]:
print("Task A accuracy after training on Multi-Task Problem: {}".format(evaluate(multi_task_model, task_A_test)))
print("Task B accuracy after training on Multi-Task Problem: {}".format(evaluate(multi_task_model, task_B_test)))

Task A accuracy after training on Multi-Task Problem: 0.9727973937988281
Task B accuracy after training on Multi-Task Problem: 0.9720141887664795


In [35]:
basic_cl_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

basic_cl_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

basic_cl_model.fit(task_A_train, epochs=6)

Epoch 1/6
922/922 [==============================] - 2s 3ms/step - loss: 0.1325 - accuracy: 0.9618
Epoch 2/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0522 - accuracy: 0.9846
Epoch 3/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0328 - accuracy: 0.9904
Epoch 4/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0218 - accuracy: 0.9943
Epoch 5/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0149 - accuracy: 0.9962
Epoch 6/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0103 - accuracy: 0.9974


In [36]:
print("Task A accuracy after training model on only Task A: {}".format(evaluate(basic_cl_model, task_A_test)))

Task A accuracy after training model on only Task A: 0.9884287714958191


In [37]:
basic_cl_model.fit(task_B_train, epochs=6)

Epoch 1/6
954/954 [==============================] - 3s 3ms/step - loss: 0.1850 - accuracy: 0.9512
Epoch 2/6
954/954 [==============================] - 2s 3ms/step - loss: 0.0617 - accuracy: 0.9813
Epoch 3/6
954/954 [==============================] - 3s 3ms/step - loss: 0.0410 - accuracy: 0.9877
Epoch 4/6
954/954 [==============================] - 3s 3ms/step - loss: 0.0291 - accuracy: 0.9918
Epoch 5/6
954/954 [==============================] - 2s 3ms/step - loss: 0.0207 - accuracy: 0.9947
Epoch 6/6
954/954 [==============================] - 3s 3ms/step - loss: 0.0146 - accuracy: 0.9967


In [38]:
print("Task B accuracy after training trained model on Task B: {}".format(evaluate(basic_cl_model, task_B_test)))
print("Task A accuracy after training trained model on Task B: {}".format(evaluate(basic_cl_model, task_A_test)))

Task B accuracy after training trained model on Task B: 0.9865983724594116
Task A accuracy after training trained model on Task B: 0.2884693443775177


In [39]:
def l2_penalty(theta, theta_A):
  penalty = 0
  for i, theta_i in enumerate(theta):
    _penalty = tf.math.reduce_sum((theta_i - theta_A[i]) ** 2)
    penalty += _penalty
  return 0.5*penalty

def train_with_l2(model, task_A_train, task_B_train, task_A_test, task_B_test, epochs=6):
  # First we're going to fit to task A and retain a copy of parameters trained on Task A
  model.fit(task_A_train, epochs=epochs)
  theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}

  print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_test)))

  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

  for epoch in range(epochs):
    accuracy.reset_states()
    loss.reset_states()
    for batch, (imgs, labels) in enumerate(task_B_train):
      with tf.GradientTape() as tape:
        preds = model(imgs)
        total_loss = model.loss(labels, preds) + l2_penalty(model.trainable_variables, theta_A)
      grads = tape.gradient(total_loss, model.trainable_variables)
      model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      
      accuracy.update_state(labels, preds)
      loss.update_state(labels, preds)
      print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
          epoch+1, batch+1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
         )
    print("")
  
  print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_test)))
  print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_test)))

In [40]:
l2_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

l2_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

train_with_l2(l2_model, task_A_train, task_B_train, task_A_test, task_B_test)

Epoch 1/6
922/922 [==============================] - 2s 3ms/step - loss: 0.1345 - accuracy: 0.9614
Epoch 2/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0552 - accuracy: 0.9833
Epoch 3/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0359 - accuracy: 0.9893
Epoch 4/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0243 - accuracy: 0.9930
Epoch 5/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0168 - accuracy: 0.9955
Epoch 6/6
922/922 [==============================] - 2s 3ms/step - loss: 0.0115 - accuracy: 0.9968
Task A accuracy after training on Task A: 0.985180675983429
Epoch: 1, Batch: 954, Loss: 4.375, Accuracy: 0.538
Epoch: 2, Batch: 954, Loss: 4.283, Accuracy: 0.545
Epoch: 3, Batch: 954, Loss: 4.274, Accuracy: 0.546
Epoch: 4, Batch: 954, Loss: 4.274, Accuracy: 0.546
Epoch: 5, Batch: 954, Loss: 4.271, Accuracy: 0.546
Epoch: 6, Batch: 954, Loss: 4.270, Accuracy: 0.546
Task B accuracy after training trained m

In [51]:
def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
  task_set = task_set.repeat()
  precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

  for i, (imgs, labels) in enumerate(task_set.take(num_batches)):
    # We need gradients of model params
    with tf.GradientTape() as tape:
      # Get model predictions for each image
      preds = model(imgs)
      # Get the log likelihoods of the predictions
      ll = tf.nn.log_softmax(preds)
    # Attach gradients of ll to ll_grads
    ll_grads = tape.gradient(ll, model.trainable_variables)
    # Compute F_i as mean of gradients squared
    for i, g in enumerate(ll_grads):
      precision_matrices[i] += tf.math.reduce_mean(g ** 2, axis=0) / num_batches

  return precision_matrices

def compute_elastic_penalty(F, theta, theta_A, alpha=25):
  penalty = 0
  for i, theta_i in enumerate(theta):
    _penalty = tf.math.reduce_sum(F[i] * (theta_i - theta_A[i]) ** 2)
    penalty += _penalty
  return 0.5*alpha*penalty

def ewc_loss(labels, preds, model, F, theta_A):
  loss_b = model.loss(labels, preds)
  penalty = compute_elastic_penalty(F, model.trainable_variables, theta_A)
  return loss_b + penalty

def train_with_ewc(model, task_A_set, task_B_set, task_A_test, task_B_test, epochs=3):
  # First we're going to fit to task A and retain a copy of parameters trained on Task A
  model.fit(task_A_set, epochs=epochs)
  theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}
  # We'll only compute Fisher once, you can do it whenever
  F = compute_precision_matrices(model, task_A_set, num_batches=1000)

  print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_test)))

  # Now we set up the training loop for task B with EWC
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

  for epoch in range(epochs*3):
    accuracy.reset_states()
    loss.reset_states()

    for batch, (imgs, labels) in enumerate(task_B_set):
      with tf.GradientTape() as tape:
        # Make the predictions
        preds = model(imgs)
        # Compute EWC loss
        total_loss = ewc_loss(labels, preds, model, F, theta_A)
      # Compute the gradients of model's trainable parameters wrt total loss
      grads = tape.gradient(total_loss, model.trainable_variables)
      # Update the model with gradients
      model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      # Report updated loss and accuracy
      accuracy.update_state(labels, preds)
      loss.update_state(labels, preds)
      print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
          epoch+1, batch+1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
         )
    print("")

  print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_test)))
  print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_test)))

In [54]:
ewc_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(5)
])

ewc_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

train_with_ewc(ewc_model, task_A_train, task_B_train, task_A_test, task_B_test)

Epoch 1/3
922/922 [==============================] - 2s 2ms/step - loss: 0.1972 - accuracy: 0.9395
Epoch 2/3
922/922 [==============================] - 2s 2ms/step - loss: 0.0981 - accuracy: 0.9706
Epoch 3/3
922/922 [==============================] - 2s 2ms/step - loss: 0.0760 - accuracy: 0.9772
Task A accuracy after training on Task A: 0.9807145595550537
Epoch: 1, Batch: 954, Loss: 9.546, Accuracy: 0.201
Epoch: 2, Batch: 954, Loss: 7.050, Accuracy: 0.361
Epoch: 3, Batch: 954, Loss: 4.001, Accuracy: 0.534
Epoch: 4, Batch: 954, Loss: 3.417, Accuracy: 0.601
Epoch: 5, Batch: 954, Loss: 3.273, Accuracy: 0.635
Epoch: 6, Batch: 954, Loss: 3.191, Accuracy: 0.660
Epoch: 7, Batch: 954, Loss: 3.128, Accuracy: 0.679
Epoch: 8, Batch: 954, Loss: 3.072, Accuracy: 0.693
Epoch: 9, Batch: 954, Loss: 3.021, Accuracy: 0.704
Task B accuracy after training trained model on Task B: 0.7087110877037048
Task A accuracy after training trained model on Task B: 0.7677628993988037
